# An example of using Dask with on disk sparse strage


This example can be run in the following environement,
```
conda create -n dask-env -c conda-forge dask==0.15.1 joblib==0.11 scipy==0.19.1 jupyter python=3.6
source activate dask-env
pip install sparse
```

In [1]:
import numpy as np
import scipy.sparse

import dask.array as da
from dask.delayed import delayed
from sparse import COO
import joblib

## 1. Creating a sparse dataset on disk

We will create a random sparse array in COO format with the [sparse](https://github.com/mrocklin/sparse) library, and save it on disk with joblib 

In [2]:
np.random.seed(42)


X_sp = scipy.sparse.random(1000, 100000, format='coo')

In [3]:
X_coo = COO.from_scipy_sparse(X_sp)

In [4]:
X_coo

<COO: shape=(1000, 100000), dtype=float64, nnz=1000000>

In [5]:
X_coo.coords, X_coo.data

(array([[  867,    36,   282, ...,   132,   700,   752],
        [65682, 56755, 56882, ..., 63860, 91667,  9016]], dtype=uint32),
 array([ 0.64711429,  0.61256173,  0.68260957, ...,  0.18168748,
         0.3841033 ,  0.1567458 ]))

The `X` sparse array consist of three numpy arrays (see `sparse` package documentation). 

Now let's save this array on disk with joblib.

In [6]:
joblib.dump(X_coo, 'X_coo.pkl')

['X_coo.pkl']

## 2. Loading the array on disk

We will load the saved array with the `mmap_mode='r'` which prevents the data from being loaded in memory, unless that is explicitly requested.

In [7]:
X = joblib.load('X_coo.pkl', mmap_mode='r')

In [8]:
X

<COO: shape=(1000, 100000), dtype=float64, nnz=1000000>

In [9]:
X.coords, X.data

(memmap([[  867,    36,   282, ...,   132,   700,   752],
         [65682, 56755, 56882, ..., 63860, 91667,  9016]], dtype=uint32),
 memmap([ 0.64711429,  0.61256173,  0.68260957, ...,  0.18168748,
          0.3841033 ,  0.1567458 ]))

Unlike the original array, `X` is not loaded in memory, and slices on `X` will lazilly load requred data from disk, 

In [10]:
sl = (slice(100), slice(10000))
X[sl], X[sl].data

(<COO: shape=(100, 10000), dtype=float64, nnz=10066>,
 array([ 0.30196698,  0.12631232,  0.13893607, ...,  0.62841608,
         0.52617956,  0.59128451]))

A slice of `X` is an actual array, not a memory map.

## 3. Trying to use dask with the on disk array

### 3.1 Calling the from_array on the on disk array

In [11]:
ds = da.from_array(X, chunks=(1000))
ds.compute()

IndexError: tuple index out of range

### 3.2 Calling the from_array on the in memory array

Also produces the same error as above,

In [12]:
ds = da.from_array(X_coo, chunks=(1000))
ds.compute()

IndexError: tuple index out of range

### 3.3 Calling the from_delayed on the on disk array

In [23]:
def gen_batches(n, batch_size):
    """Batches generator from sklearn.utils
    """
    start = 0
    for _ in range(int(n // batch_size)):
        end = start + batch_size
        yield slice(start, end)
        start = end
    if start < n:
        yield slice(start, n)

def _get_sparse_slice(fname, sl):
    return joblib.load(fname, mmap_mode='r')[sl]
        

delayed_list = [delayed(_get_sparslice)('X_coo.pkl',sl)
                for sl in gen_batches(X.shape[0], 100)]

In [24]:
da.from_delayed(delayed_list, shape=X.shape, dtype=X.dtype)

AttributeError: 'list' object has no attribute 'key'

### 3.4 Allocating an on disk dense array, loading it as mmap + in memory replacement

This appears to be the only apprach that works out of the box,

In [15]:
fp = np.memmap('X_dense.npy', dtype='float64', mode='w+', shape=X.shape)

In [16]:
# Copy the data from the sparse to the dense array
for sl in gen_batches(X.shape[0], 10):
    fp[sl] = X_coo[sl].todense()

In [17]:
ds = da.from_array(fp, chunks=(100))
ds.map_blocks(COO)

dask.array<COO, shape=(1000, 100000), dtype=float64, chunksize=(100, 100)>

In [18]:
ds.sum().compute()

500072.39047789795

## Conclusions

Given a sparse array saved on disk, it appears that currently the only solution is to pre-alocate a dense array on disk with the same shape, copy the data from the sparse array to the dense array, open it with `dask.array.from_array` then do an in memory replacement from dense to sparse. This is terribly inefficient, and ideas on how to avoid this double conversion are very welcome...